In [1]:
import csv
import os
import json
import codecs
import re
import pandas as pd
import xlrd

In [2]:
posts_data = pd.read_excel('topReactedTweets.xlsx')

In [3]:
posts_data.head()

,text,nbr_retweet,nbr_favorite,nbr_reply,reaction,ID,date
0,# BTSARMY is jumping into what's happening in ...,13314,17058,340,30712,1025888565048365056,2018-08-05 05:37:48
1,"Students in Bangladesh are being attacked, mur...",14861,6634,68,21563,1025870475992358912,2018-08-05 04:25:55
2,Praying for Bangladesh students safety #WeWant...,9910,7320,32,17262,1026074162438131713,2018-08-05 17:55:18
3,All of us know the power of social media and w...,9769,7256,54,17079,1025812106925539329,2018-08-05 00:33:59
4,Students in Bangladesh are just unstoppable. Y...,8477,8012,71,16560,1026160890779844608,2018-08-05 23:39:56


In [4]:
len(posts_data)

2653

In [5]:
reactions = list(posts_data['reaction'])

In [6]:
import numpy as np
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
np.median(np.array(reactions))

62.0

In [8]:
data = posts_data[posts_data['reaction'] > 62].copy()

In [9]:
data.tail()

,text,nbr_retweet,nbr_favorite,nbr_reply,reaction,ID,date
1318,Excellent @Channel4News report by @alextomo on...,17,43,3,63,1034918187673047053,2018-08-30 03:38:18
1319,Did you say 'only males'? Our ladies are equal...,10,52,1,63,1025069522561056768,2018-08-02 23:23:13
1320,পুলিশ তুমি মানুষ হতে শিখো ৷ হিংস্রতো জানোয়ার ...,4,56,3,63,1024979167945809922,2018-08-02 17:24:11
1321,অান্দোলনকারি শিক্ষার্থী ভাই-বোনেরা__ যদি ছাত্র...,3,60,0,63,1026313373007265793,2018-08-06 09:45:50
1322,শিক্ষার্থীদের উপর হামলার প্রতিবাদে জাহাঙ্গীরনগ...,2,59,2,63,1026152573185585153,2018-08-05 23:06:53


In [10]:
data['text'].fillna('',inplace = True)

In [11]:
txt_lst = list(data['text'])

In [20]:
for i in range(len(txt_lst)):
    try:
        for j in range(len(txt_lst[i])):
            if (ord(txt_lst[i][j]) >127):
                txt_lst[i] = txt_lst[i].replace(txt_lst[i][j],'')
    except:
        continue

In [21]:
txt_lst

["# BTSARMY is jumping into what's happening in Bangladesh . The hashtag #WeWantJustice is #BTS fans' way of creating more attention on the issue. Here's a report on the situation. https:// abcnews.go.com/International/ wireStory/scores-injured-traffic-protests-bangladesh-capital-57030231   https:// twitter.com/hopeworldmono/ status/1025870887881453568  ",
 'Students in Bangladesh are being attacked, murdered, and r* for standing up against their government. They have now had their wifi cut-off to prevent the news from spreading around. Please do not silence them. #WeWantJustice pic.twitter.com/b1CpACqkh4',
 'Praying for Bangladesh students safety #WeWantJustice pic.twitter.com/YxxOTWhxBP',
 'All of us know the power of social media and we all have a voice on this platform. Now is the time to use it for good. Please help spread awareness about the injustice happening in Bangladesh . #WeWantJustice https:// twitter.com/BBCWorld/statu s/1025764029892096001  ',
 "Students in Bangladesh ar

In [22]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

In [23]:
new_data = []

for i in range (len(txt_lst)):
    clean_tweet = re.match('(.*?)http.*?\s?(.*?)', txt_lst[i])
    if clean_tweet:
        new_tweet = ''+clean_tweet.group(1) + clean_tweet.group(2)
        
        new_data.append(new_tweet)
    else:
        new_data.append(txt_lst[i])

txt_lst = new_data

In [24]:
txt_lst = [re.sub('\S*@\S*\s?', '', sent) for sent in txt_lst]

# Remove new line characters
txt_lst = [re.sub('\s+', ' ', sent) for sent in txt_lst]

# Remove distracting single quotes
txt_lst = [re.sub("\'", "", sent) for sent in txt_lst]

pprint(txt_lst[:1])

['# BTSARMY is jumping into whats happening in Bangladesh . The hashtag '
 '#WeWantJustice is #BTS fans way of creating more attention on the issue. '
 'Heres a report on the situation. ']


In [25]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(txt_lst))

print(data_words[:1])


[['btsarmy', 'is', 'jumping', 'into', 'whats', 'happening', 'in', 'bangladesh', 'the', 'hashtag', 'wewantjustice', 'is', 'bts', 'fans', 'way', 'of', 'creating', 'more', 'attention', 'on', 'the', 'issue', 'heres', 'report', 'on', 'the', 'situation']]


In [26]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['btsarmy', 'is', 'jumping', 'into', 'whats_happening', 'in', 'bangladesh', 'the', 'hashtag', 'wewantjustice', 'is', 'bts', 'fans', 'way', 'of', 'creating', 'more', 'attention', 'on', 'the', 'issue', 'heres', 'report', 'on', 'the', 'situation']


In [27]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [28]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['jump', 'whats_happene', 'bangladesh', 'hashtag', 'wewantjustice', 'bts', 'fan', 'way', 'create', 'attention', 'issue', 'here', 'report', 'situation']]


In [29]:
res = data_lemmatized
csvfile = "C:\\Users\\Alvi Rahman\\Desktop\\Farhana Miss\\student\student2\\New folder\\data_lem_reaction.txt"

#Assuming res is a list of lists
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(res)